# Chit Chat Chatbots

In the previous lab, we explored models that try to answer questions by reasoning over free-text input. In this lab, we will explore two types of models to create chatbots.

First, let's consider important qualities for a chit-chat chatbot system


1.   **Readability** - whatever model we use, the chats it creates should be easily understood by humans
2.   **Consistency** - when chatting with a chatbot, the bot should maintain consistent information. Imagine a bot that says "Hi I'm Jack'' and then "Hello, my name is Jane" - quite confusing
3.    **Engaging** - To encourage users to talk to the bot, the bot should be able to generate interesting, engaging responses. If the only response was "wow, that's cool," users are quite unlikely to want to talk very much to the chat bot



In [1]:
%%html
<p style='color: blue;'>
  Throughout the lab, there will be <b>questions</b> you should answer. <b>All questions you need to write an answer to will be in this blue color.</b>
  
  <br>Please write brief answers- no need for long explanations. 
  <br>There can be multiple correct answers to the questions.
  
  <br><br>The goal of these questions is to:
  <ul style='color: green;'>
    <li> Review the lecture material in the context of practical models and develop intuition about the models
    <li> Develop a sense of experimentation - we will pretend we have a dataset and will walk through an experimental thought process.
  </ul>

<b>We are going to do the lab as a group. <br>I will explain the sections in more depth, as we did not cover dialogue deeply during the lecture. <br> After we discuss, I will provide time for you to write a few sentences. At the end of the lab, you will hand it in. In theory, everyone should be finished together!</b>

  
</p>

## Data

The dataset we will use for this lab is called `PersonaChat` - it was created to directly address problem 2. Each person talking in the dataset has a personality, which helps maintain consistency in the dialogue.

In [2]:
!git clone https://github.com/facebookresearch/ParlAI.git ~/ParlAI
!cd ~/ParlAI && git checkout convai2archive
!cd ~/ParlAI; python setup.py develop

Cloning into '/root/ParlAI'...
remote: Enumerating objects: 39179, done.
remote: Counting objects: 100% (434/434), done.
remote: Compressing objects: 100% (321/321), done.
remote: Total 39179 (delta 177), reused 314 (delta 107), pack-reused 38745
Receiving objects: 100% (39179/39179), 64.98 MiB | 32.02 MiB/s, done.
Resolving deltas: 100% (27910/27910), done.
Note: checking out 'convai2archive'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at 8c79dac4 add documentation for MEssages (#2339)
running develop
running egg_info
creating parlai.egg-info
writing parlai.egg-info/PKG-INFO
writing dependency_links to parlai.egg-info/

**Example: **

your persona: i just started college.

your persona: i have 3 science classes.

your persona: i work part time in the campus library.

your persona: i am living at home but hope to live in the dorms next year.

**Partner Dialogue**: hi how are you doing

**Your Response**: great ! just got off work and relaxing before i study

In [4]:
# let's download and take a look at some examples of data in PersonaChat
!python ~/ParlAI/examples/display_data.py --task personachat --datatype train

[ optional arguments: ] 
[  display_ignore_fields: agent_reply ]
[  max_display_len: 1000 ]
[  num_examples: 10 ]
[ Main ParlAI Arguments: ] 
[  batchsize: 1 ]
[  datapath: /root/ParlAI/data ]
[  datatype: train ]
[  download_path: /root/ParlAI/downloads ]
[  hide_labels: False ]
[  image_mode: raw ]
[  init_opt: None ]
[  multitask_weights: [1] ]
[  numthreads: 1 ]
[  show_advanced_args: False ]
[  task: personachat ]
[ ParlAI Model Arguments: ] 
[  dict_class: None ]
[  init_model: None ]
[  model: None ]
[  model_file: None ]
[ PytorchData Arguments: ] 
[  batch_length_range: 5 ]
[  batch_sort_cache_type: pop ]
[  batch_sort_field: text ]
[  numworkers: 4 ]
[  pytorch_context_length: -1 ]
[  pytorch_datapath: None ]
[  pytorch_include_labels: True ]
[  pytorch_preprocess: False ]
[  pytorch_teacher_batch_sort: False ]
[  pytorch_teacher_dataset: None ]
[  pytorch_teacher_task: None ]
[  shuffle: False ]
[ ParlAI Image Preprocessing Arguments: ] 
[  image_cropsize: 224 ]
[  image_siz

In [5]:
%%html
<p style='color: blue;'>
  <b>Questions:</b>
  <ul style='color: blue;'>
    <li>What do the personalities look like?</li>
    <ul><font color="green">They describe who the bots are, what they do, where they work, ..etc.</font></ul>
    <li>How does creating bots with these simple personalities address consistency for chatbots? </li>
    <ul><font color="green">By guiding the behaviour of the model to follow the characteristics of the bot.</font></ul>
    <li>What are some drawbacks/limitations of these specific personalities for addressing the problem of consistency?</li>
    <ul><font color="green">They can’t fully describe whta a bot is, resulting in a narrow model that doesn’t genralize.</font></ul>
  </ul>
</p>



---


***Let's understand how much data we have. Let's compute the following using ParlAI:***


1.   **How many turns of data do we have?** In dialogue datasets, "amount of data" is measured in dialogue turns. Each time there is a single line of dialogue, that is called a "turn"
2.   **On average, how many words form a model input?**


---




In [6]:
!python ~/ParlAI/parlai/scripts/data_stats.py -t personachat -dt train -ltim 10000

[ note: changing datatype from train to train:ordered ]
[creating task(s): personachat]
[loading fbdialog data:/root/ParlAI/data/Persona-Chat/personachat/train_self_original.txt]
[ loaded 8939 episodes with a total of 65719 examples ]
15s elapsed: {'exs': 65719, '%done': '100.00%', 'time_left': '0s', 'stats': '
input:
   utterances: 65719
   avg utterance length: 18.356974390967604
   tokens: 1206402
   unique tokens: 14209
   unique utterances: 64580
labels:
   utterances: 65719
   avg utterance length: 11.929411585690591
   tokens: 783989
   unique tokens: 14507
   unique utterances: 64119
both:
   utterances: 131438
   avg utterance length: 15.143192988329098
   tokens: 1990391
   unique tokens: 18741
   unique utterances: 128197
'}


## Evaluation

How are dialogue models evaluated?



1.   **Automatic Evaluation**: Hits @ 1, Hits @ 5, Hits @ 10, F1
2.   **Human Evaluation**: Pairwise Comparison, Human Rating



In [7]:
%%html
<p style='color: blue;'>
  <b>Questions:</b>
  <ul style='color: blue;'>
    <li>Take some notes about what these metrics are and what they mean here</li>
    <ul style='color: green;'>

      <li>Automatic Evaluation</li>
      <ul>
      <li>Hits @1: Accuracy</li>
      <li>Hits @5: The number of times that the model ranked the correct answer in its first 5 answers.</li>
      <li>Hits @10: The number of times that the model ranked the correct answer in its first 10 answers</li>
      <li>F1: (2*precision * recall) /( precision + recall)</li>
      </ul>

      <li>Human Evaluation</li>
      <ul>
      <li>Pairwise Comparison: The human is given two conversation and he/she must decide which one is better than the other.</li>
      <li>Human Rating: Giving a score rating fluency, engagement .. etc. This metric is subjective and vary too much, and less accurate thatn PC.</li>
      </ul>

    </ul>
  </ul>
</p>


## Models

There are two main kinds of dialogue models. 

*Retrieval* Models analyze the current dialogue context and try to find appropriate responses in the dataset.

*Generative* Models analyze the current dialogue context
and try to write an answer, word by word, from left to right.
This can be thought of as an application of sequence-to-sequence models,  where the "encoder side" is the dialogue history and the "decoder side" is the dialogue response your chatbot should generate.

In [25]:
%%html
<p style='color: blue;'>
  <b>Questions:</b>
  <ul style='color: blue;'>
    <li>Let's discuss the pros/cons of retrieval compared to generative models - Are there settings when you might want to use one over the other?</li>
      <ul style='color: black;'>
        <li>Retrieval Moels <font color="green">pros</font>/<font color="red">cons</font>:</li>
        <ul>
          <font color="green"><li>More flunect/diverse/controllable answers.</li></font>
          <font color="red"><li>Not generalize.</li></font>
        </ul>

      <li>Generative Models <font color="green">pros</font>/<font color="red">cons</font>:</li>
        <ul>
          <font color="green"><li>Generalized/Engaging.</li></font>
          <font color="red"><li>Need a lot of data.</li></font>
        </ul>

       <li>Situation to use retrieval over generative:</li>
        <ul>
          <li>Restrictive application --> Prefer Retrival Models.</li>
        </ul>

        
      </ul>
    <li>Compare a chit-chat application to something like booking a movie ticket- would you want to use generative, retrieval, or something else to accomplish that task? Why?</li>
      <ul style='color: black;'>
        <li>Domain: Open.</li>
        <li>Goal: Engage with users.</li>
        <li>Open doamin --> Restricted --> Retrieval Model.</li>
      </ul>
    <li>How can you evaluate generative models with the metrics we discussed before? How do you think they will perform compared to retrieval models?</li>
      <ul style='color: black;'> 
        <li>Evaluation metrics:</li>
        <ul>
          <li>Automatic evaluation: F1, BLEU --> They won't give a good evaluation because the space of answers is so huge.</li>
          <li>Human evaluation: Gives a good estimation of performance.</li>
        </ul> 
        <li>Generalize better than Retrieval Models.</li>
      </ul>
    <li>In lecture, Antoine mentioned issues with generative model generation being generic and short. How does this happen in beam search?</li>
      <ul style='color: black;'>
        <li>Beam search pick sequences with the highest probability, which for most of scenarios are generic and short.</li>
      </ul>
</ul>

<b> If you would like me to discuss how to actually use generative models in dialog, please say something! Otherwise, we will skip.</b>
</p>

### Retrieval Models

Let's train a model to do retrieval first. We will try the *Memory Net.* 

In [27]:
# We can train a model with the following command: 
# !python ~/ParlAI/examples/train_model.py -m kv_memnn -t personachat -dt train -veps 0.25 --model-file persona_chat_retrieval_model -vmt accuracy

# but we have limited time in the tutorial, so let's use an already pretrained model

Quick Parameter Refresher:


*  `-m ` means which model we're going to use. Recall retrieval models are trained to rank the true response higher over a set of potential responses from the dataset (in ParlAI, these are called the "label candidates"). When it's time to write a dialogue response, the retrieval model returns the response that is ranked the highest
*  -`t` refers to the task. Here, we are training on PersonaChat data.
* `-dt` refers to the data split. We want to train our model, so we are using the training set.
* `-veps` refers to how often we should evaluate during training, our performance on validation. recall this is important because models, particularly neural ones, have the capacity to memorize the training dataset. So it's important to check how the model is doing on the validation set.
* `--model-file` refers to when your model is saved, what should the filename be
*  `-vmt` refers to the metric which we'll use to decide which model is the best. We'll cover this in the next section





**Let's interact with the model to get a sense of what it's learning. **How is this chat going to work?



1.   You will be assigned a persona. You will chat to the model by typing in the chat box.
2.   The chatbot also has a persona. It's secret and hidden from you!
3.   When you've finished chatting with this bot, type [DONE] and a new model persona will be assigned to the bot, so you can talk to a new bot. 
4.   When you move on to the next chatbot persona, the previous persona will be revealed. 

Interact with the chatbots and the personas. **Try to think about the following:**

*   Do the chatbots follow their persona a lot?
*   Was it difficult to follow your persona?





In [28]:
!python ~/ParlAI/projects/convai2/interactive.py -mf models:convai2/kvmemnn/model

[building data: /root/ParlAI/data/models/convai2/kvmemnn/kvmemnn.tgz]
[ downloading: http://parl.ai/downloads/_models/convai2/kvmemnn.tgz to /root/ParlAI/data/models/convai2/kvmemnn/kvmemnn.tgz ]
unpacking kvmemnn.tgz
[ warning: overriding opt['model_file'] to /root/ParlAI/data/models/convai2/kvmemnn/model (previously: /checkpoint/jase/20180328/kvmemnn_sweep10/persona-self_rephraseTrn-True_rephraseTst-False_lr-0.1_esz-2000_margin-0.1_tfidf-False_shareEmb-True_hops1_lins0/model )]
[ creating KvmemnnAgent ]
Dictionary: loading dictionary from /root/ParlAI/data/models/convai2/kvmemnn/model.dict
[ num words =  19153 ]
Loading existing model params from /root/ParlAI/data/models/convai2/kvmemnn/model
[loading candidates: /root/ParlAI/data/models/convai2/kvmemnn/model.candspair*]
[caching..]
=init done=
/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.form

In [30]:
%%html
<p style='color: blue;'>
  <b>Questions:</b>
  <ul style='color: blue;'>
    <li>What does this model seem to be doing well? What is it doing poorly? </li>
    <li>Why might it be performing poorly? What kind of experiment could you design to test your hypothesis?</li>
    <li>How do we know if we need to use a more complex model? Would we always want to use a more complex model? Why or why not?</li>
  </ul>
</p>

In [31]:
# Here is a command to train a Transformer Ranker model if you would like to try it out
# !python ~/ParlAI/examples/train_model.py -m transformer/ranker -t personachat -dt train -veps 0.25 --model-file persona_chat_retrieval_model -vmt accuracy



---


An important aspect of training models is analyzing them. ***Try to answer the following questions.*** 

---




In [34]:
%%html
<p style='color: blue;'>
  <b>Questions:</b>
  <ul style='color: blue;'>
    <li>Are the models using the persona that we have provided? How can you tell? If I asked you to prove it to me, what experiments could you conduct? </li>
      <ul style='color: black;'>
        <li>
          Yes. From talking to it above I can see a lot of keywords related to its persona.
        </li>

        <li>
          Experiment: Ask the bot specific question about its persona, and see if it will reply using the persona or no.
        </li>
      </ul>
    <li>Previously, we computed some statistics about how long the persona is in the training data. The model has also only seen words present in the training dataset. But what happens if you push the model outside of what data it's been trained on? What kind of performance do you get? Why does this happen, and what could you do if you wanted to improve the model's ability to generalize? </li>
    <ul style='color: black;'>
      <li>For retrieval model this push it to reply unconsistenly</li>
      <li>Trainning in a bigger corpuse will improve the model generalizability.</li>
    </ul>
    <li>In ParlAI, we've set the parameters to save the model's best performance based on validation accuracy. What would happen if we saved the model based on the best training accuracy? Why does this happen? (if you like, try this out on your own and see the effect when you interact with the bot)</li>
    <ul style='color: black;'>
      <li>
        This will cause OVERFITTING.
      </li>
    </ul>
  </ul>

<b> If you would like me to discuss how to use BERT in dialog, please say something! Otherwise, we will skip.</b>
</p>

### [for self exploration] Generative Models

Generative models must produce word for word what they are going to say next in the dialogue. When predicting the next word, it produces a probability distribution over the entire vocabulary space for which word to generate next. To reduce the vocabulary space, we will use **byte-pair encoding** (BPE). 

*How does BPE work?* The BPE algorithm takes as input the training data and the number of *operations* it can do. It passes over the training set and tries to create sub-word units. For example, the word "beautiful" might be split into "beau" "ti" "ful". Each time it splits a word into sub-words, that is one operation. The final vocabulary output consists of these subwords. So "ful" can be part of "beautiful" and part of "fruitful" and so on.


**Questions to ask yourself**:


1.   Why is it important to keep the vocabulary space small?
2.   What does perplexity measure? Why would we use it as a training objective? 




In [35]:
# !python ~/ParlAI/examples/train_model.py -m transformer/generator -t personachat -dt train -veps 0.25 --model-file persona_chat_generative_model -vmt ppl

## Final Thoughts

**What did we learn about dialogue modeling? Review Questions to ask yourself**

*   How do retrieval models work? What about generative? What are their pros and cons?
*   What are some important traits of dialogue systems? How might the traits differ for different dialogue tasks?


**General Takeaways about Machine Learning and Experimentation:**

*   We don't try models just to try them - try to have a reason for conducting an experiment. As we did in the lab, try to analyze what's working well in your models and working poorly. Try to use these reasons to guide why you might want to try other models. Complex is not necessarily better. 
*   Certain models can be better for certain tasks. As we've seen, generative models are working really well for tasks such as machine translation, but have a bit to go before becoming general purpose dialogue generators. 



**I'm really interested in dialogue! What can I do to learn more?**


*   Play around in ParlAI: ParlAI is a general library with many great dialogue models and code for them. It also provides a standard interface to access datasets and interact with various models. 
*   Read the PersonaChat Paper: https://arxiv.org/pdf/1801.07243.pdf
*   Dialog using knowledge: One challenge of these chit chat systems is they do not concretely know any facts. So if you want to chat about a specific topic, the models cannot produce any relevant information - they say generic utterances or incorrect facts. One way to remedy this is to incorporate **knowledge** into the dialogue agents. This has been investigated in many different ways, but one of the first papers to show this is https://arxiv.org/abs/1811.01241. In this work, data is collected by asking one speaker to reference Wikipedia sentences.
* Dialog with BERT: pretty new,  there is an investigation of two ways to use BERT in this paper: https://arxiv.org/abs/1903.03094. 


